In [17]:
import os
import sqlite3
import pandas as pd

pd.options.plotting.backend = "plotly"

path = r'H:\Desktop1123\Learning\09 - Python Projects\02 FitBot\FitBot'
os.chdir(path)

In [49]:
conn = sqlite3.connect('data/database.db')
c = conn.cursor()

df = (
    pd.read_sql_query("SELECT * FROM fitbit", conn)
      .assign(
                dateTime=lambda x: pd.to_datetime(x.dateTime),
                scrapeTime=lambda x: pd.to_datetime(x.scrapeTime)
      )
)

(
    df.loc[df.scrapeTime == df.groupby('dateTime').scrapeTime.transform('max')]
    .sort_values('dateTime',ignore_index=True,ascending=True)
    .assign(delta = lambda x: x.weight.diff())
    .assign(ddelta = lambda x: x.delta.diff())
    .melt(id_vars=['dateTime'],value_vars=['weight','delta','ddelta'],var_name='variable',value_name='value')
    .plot(x='dateTime', y='value', facet_row = 'variable',title='Weight Change Over Time',kind = 'scatter',trendline='rolling',trendline_options=dict(window=7),height = 1000)
    .update_yaxes(matches=None)
)


In [46]:
(
    df.loc[df.scrapeTime == df.groupby('dateTime').scrapeTime.transform('max')]
    .sort_values('dateTime',ignore_index=True,ascending=True)
    .assign(delta = lambda x: x.weight.diff())
    # .hist(x='delta',title='Weight Change Distribution',bins=30)
    
)

,scrapeTime,dateTime,steps,weight,calories,carbs,fat,protein,sodium,delta
0,2025-01-04 16:01:30,2023-12-05,1771,78.4,0,0,0,0,0,NaN
1,2025-01-04 16:01:30,2023-12-06,4075,78.2,0,0,0,0,0,-0.200
2,2025-01-04 16:01:30,2023-12-07,11311,78.4,0,0,0,0,0,0.200
3,2025-01-04 16:01:30,2023-12-08,5409,78.5,0,0,0,0,0,0.100
4,2025-01-04 16:01:30,2023-12-09,8056,78.7,0,0,0,0,0,0.200
...,...,...,...,...,...,...,...,...,...,...
397,2025-01-06 09:22:14,2025-01-05,10687,81.8,2025,204,48,181,1974,0.201
398,2025-01-07 09:11:00,2025-01-06,15946,81.8,1640,130,36,191,164,0.000
399,2025-01-08 09:51:39,2025-01-07,11475,80.3,2193,186,68,203,386,-1.500
400,2025-01-09 09:00:56,2025-01-08,13595,81.5,2145,164,53,248,134,1.200
